In [1]:
import molsysmt as msm
import numpy as np
from molsysmt import pyunitwizard as puw

In [2]:
molsys = msm.convert(msm.systems['pentalanine']['traj_pentalanine.h5'], to_form='molsysmt.MolSys')
d = msm.structure.get_distances(molsys, selection=8, selection_2=18, structure_indices=2000)

In [3]:
d

Magnitude,[[[0.3874317590975941]]]
Units,nanometer


In [ ]:
coors8 = msm.get(molsys, element='atom', selection=8, structure_indices=2000, coordinates=True)[0,0]
coors18 = msm.get(molsys, element='atom', selection=18, structure_indices=2000, coordinates=True)[0,0]
box = msm.get(molsys, element='system', structure_indices=2000, box=True)[0]

In [ ]:
coors8

In [ ]:
coors18

In [ ]:
box

In [ ]:
vector = coors18-coors8

In [ ]:
np.linalg.norm(vector)

In [ ]:
np.round(-0.5)

In [ ]:
np.floor(-0.5+0.5)